In [2]:
import pandas as pd 
from sklearnex import patch_sklearn
patch_sklearn()

import tensorflow as tf

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [5]:

# 创建映射字典
category_to_number = {'装': 1, '卸': -1, 'other': 0}

# 定义一个函数，用于将Category列的文本转换为对应数值
def convert_category(row):
    v=row['state']
    if v not in category_to_number:
        return 0
    return category_to_number[v]

df =pd.read_csv('./data/states.txt')
df['start'] =df.apply(lambda row:row['date']+' '+row['start_time'],axis=1)
df['end'] = df.apply(lambda row:row['date']+' '+row['end_time'],axis=1)
df['state'] = df.apply(lambda row: row['state'].strip(),axis=1)
df[['start','end']]=df[['start','end']].apply(pd.to_datetime)
df['statev'] = df.apply(convert_category,axis=1)
df.to_csv('./data/states.csv')
df.head()

NUM_CLASSES = 3 # 10分类
labels = df['state'] # sample label
batch_size = tf.size(labels) # get size of labels : 4
labels = tf.expand_dims(labels, 1) # 增加一个维度
indices = tf.expand_dims(tf.range(0, batch_size,1), 1) #生成索引
concated = tf.concat([indices, labels] , 1) #作为拼接
# onehot_labels = tf.one_hot(concated, tf.stack([batch_size, NUM_CLASSES]), 1.0, 0.0) # 生成one-hot编码的标签

# 使用tf.one_hot直接将稀疏标签转换为one-hot编码
onehot_labels = tf.one_hot(labels, depth=NUM_CLASSES)


InvalidArgumentError: cannot compute ConcatV2 as input #1(zero-based) was expected to be a int32 tensor but is a string tensor [Op:ConcatV2] name: concat

In [ ]:
# 定义读取，处理数据函数
def fix_data(inputDF:pd.DataFrame)->pd.DataFrame:
    # 创建一个新的空列用于存储结果
    inputDF.insert(0,'state',0)
    
    # 对df1中的每一行遍历，并查找df2中符合条件的记录
    for index, row in inputDF.iterrows():
        condition = (df['start'] <= row['时间']) & (row['时间'] <= df['end'])
        match = df[condition]
        
        if not match.empty:
            # 如果找到了匹配项，则将df2的'state'赋值给df1的新列
            inputDF.at[index, 'state'] = match['statev'].values[0]
        else:
            # 如果没有找到匹配项，则保持原样（这里已经初始化为0）
            pass    
    return inputDF

def read_data(file_path:str)->pd.DataFrame:
    return fix_data( pd.read_excel(file_path,engine='openpyxl',parse_dates=['时间','轨迹时间']))

In [55]:




df17 =read_data("./data/data_17.xlsx")
df21 =read_data("./data/data_21.xlsx")
df22 =read_data("./data/data_22.xlsx")
df23 =read_data("./data/data_23.xlsx")
display(len(df17.query('state != 0')),len(df21.query('state != 0')),len(df22.query('state != 0')),len(df23.query('state != 0')))


557

340

845

1420